## Benchmarking Machine Learning Code

In this tutorial, we are going to tackle a common problem that most people often face while running their code on Borah or any other super computer out there. However, you don't have to worry, super computers are built to solve this kind of problem. We will perform a benchmarking test on some machine learning problem to demonstrate how we can take full advantage of the resources on Borah. 

### Problem Statement

- Let's assume you have a code that takes long hours to run. 
- In this case, let's say the code runs for at least 12 hours. 
- This kind of code might require more computing power than what is on your local computer. 
- So you decided to use Borah.
- You always love to write and run or test your code in a Jupyter Notebook.
- However, you only have 4 hours maximum when you Borah OnDemand and the code takes at least 12 hours to run.
- What should you do?

First of all Jupyter Notebooks are not ideal for running codes that run for several hours. As long as the code is still running, you will have to leave the notebook opened, the computer must always be on and/or always connected to the internet if the code requires an internet connection to run, etc. This means you have to basically babysit your code until it's done running. Jupyter Notebooks are good for writing and running/testing codes that do not require a lot of time to run. Let's say each cell in the notebook only takes a few seconds or minutes to run and in total it does not take several hours to run the entire notebook.

In our example above, does that mean Borah cannot help? No. Borah can definitely help because it was built for tasks like this. This is why the people at the Research Computing Department are ther to help you. They are working hard every day just to make your experiecne on Borah easier and fun. This is also why this tutorial is made available to help you. Beyond this tutorial, you can still reach out to the Research Computing department for help on Borah and/or high performance computing related matters.

### How to Solve the Problem

Let's start by considering the sample code below.

I have created a very large and deep PyTorch tutorial that:
- Uses 10 million synthetic samples with 200 features.
- Builds a 40-layer deep fully connected neural network with large hidden dimensions.
- Trains for 50 epochs, which should take 12+ hours on a personal computer, especially without a high-end GPU.
- Logs training loss and validation accuracy.


Now, let's break down the complex PyTorch tutorial you just created. This code is designed to train a massive deep neural network on synthetic data, with the goal of maximizing training time and computational load for experimentation or benchmarking.

---
- **1. The code block:**
```python
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
```
    - Chooses GPU if available, otherwise defaults to CPU.
    
---
- **2. The code block:**
```python
X, y = make_classification(n_samples=10_000_000, n_features=200, ...)
```
    - Generates 10 million samples with 200 features.
    - 150 of the features are informative (helpful for classification). These are features that actually help in distinguishing between the classes (i.e. they influence the target `y`). 
    - 30 are redundant (correlated). This means they are linear combinations of the informative features — they’re correlated with the informative ones.
    - The remaining 20 features are `noise features`. They're completely random and do not contribute to classification — they serve to make the problem more realistic by introducing irrelevant or noisy data. They help to test model robustness, simulate real-world data, which usually contains irrelevant or uninformative features, and prevent the model from overfitting to only clean, synthetic features.
    
---
- **3. The code block:**
```python
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
```
    - Splits the dataset into 90\% training and 10\% testing.

---
- **4. The code block:**
```python
class LargeDataset(Dataset):
    ...
```
    - Is a custom data set class that wraps the NumPy arrays in a PyTorch data set for use with DataLoader.
    
---
- **5. The code block:**
```python
train_loader = DataLoader(..., batch_size=2048, num_workers=4)
```
    - Creates Data Loaders and loads data in batches of 2048.
    - `num_workers=4` means parallel data loading with 4 processes (if supported). Thus, it will take a longer time to load if not supported.
    
---
- **6. The code block:**
```python
class DeepNN(nn.Module):
    ...
```
    - Defines a massive deep neural network.
    - This model is:
        - 40 layers deep, each with:
        - A Linear layer (fully connected)
        - BatchNorm1d for training stability
        - ReLU activation
        - Dropout to prevent overfitting
    - Final output: 2 classes (binary classification)
    - This is what makes the model computationally expensive.
    
---
- **7. The code block:**
```python
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
```
    - Defines loss and optimizer
    - `CrossEntropyLoss` is suitable for classification tasks.
    - `AdamW` is a variant of Adam that decouples weight decay (helps with regularization).
    
---
- **8. The code block:**
```python
for epoch in range(start_epoch, num_epochs):
    ...
```
    - Is the training loop
    - Each epoch:
        - Loops over batches
        - Sends inputs/labels to device
        - Computes outputs --> loss --> gradients --> weight updates
        - Tracks average loss per epoch
     - Also includes validation at each epoch:
        - Runs the model on the test set (no gradient tracking)
        - Calculates and stores accuracy
        
---
- **9. The code block:**
```python
start_time = time.time()
...
end_time = time.time()
```
    - Measures and prints how long training takes in hours.
    
---
- **10. The code block:**
```python
plt.plot(train_losses, ...)
```
    - Saves line plots of:
        - Training Loss vs. Epochs
        - Validation Accuracy vs. Epochs    

### Summary

| Part | Description |
|------|-------------|
| Data | 10M samples, 200 features, binary classification |
| Model | 40 hidden layers, each 2048 neurons deep |
| Training | 50 epochs, large batches (2048), uses AdamW |
| Runtime | Estimated 12+ hours on a personal CPU |
| Output | Loss & accuracy plots + final test accuracy |


In [ ]:
# Load packages
import time
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate a massive synthetic dataset (10 million samples, 200 features)
X, y = make_classification(n_samples=10e6, n_features=200, n_informative=150,
                           n_redundant=30, n_classes=2, random_state=42)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Custom dataset class
class LargeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = LargeDataset(X_train, y_train)
test_dataset = LargeDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False, num_workers=4)

# Very deep and wide neural network model
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(200, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            *[layer for _ in range(40) for layer in [
                nn.Linear(2048, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
                nn.Dropout(0.2)
            ]],
            nn.Linear(2048, 2)
        )

    def forward(self, x):
        return self.net(x)

model = DeepNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Training loop
num_epochs = 50
train_losses = []
val_accuracies = []

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    val_accuracies.append(accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Val Accuracy: {accuracy:.4f}")

end_time = time.time()
print(f"Training completed in {(end_time - start_time) / 3600:.2f} hours")

# Plot results
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.savefig('loss_plot.png')

plt.figure()
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy Over Epochs')
plt.legend()
plt.savefig('accuracy_plot.png')

### How to Improve the Code

The following improvement can be made to the code.
- Add TensorBoard logging.
- Parallelize or optimize the data pipeline.
- Turn this into a distributed training script 
- Save checkpoints for resuming later.

Now, let's navigate through a step-by-step breakdown of how we improve the code using the suggested improvements above.

---
#### Add TensorBoard Logging.
TensorBoard logging was added in the code. It appears inside the `train()` function and is scoped to `rank == 0`, so only one process (usually GPU 0) writes to TensorBoard to avoid duplication.

---
- **1. The code block:**
```python
log_dir = f"runs/ddp_rank{rank}_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir=log_dir) if rank == 0 else None
```
    - Is the TensorBoard Setup (inside `train()`).
    
- **2. The code block:**
```python
if rank == 0:
    writer.add_scalar("Loss/train", avg_loss, epoch)
    writer.add_scalar("Accuracy/val", accuracy, epoch)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Val Accuracy: {accuracy:.4f}")
```
    - Logging metrics after each epoch:
    
- **3. The code block:**
```python
if rank == 0:
    writer.close()
```
    - Closes the writer after training
    - What It Logs:
        - `Loss/train`: The average training loss per epoch.
        - `Accuracy/val`: The validation accuracy per epoch.
    - You can visualize it by running the code below in terminal and then open your browser to [http://localhost:6006](http://localhost:6006).
    ```bash
    tensorboard --logdir=runs
    ```

#### Parallelize or Optimize the Data Pipeline.
The data pipeline is now optimized for performance! Here's what was added:
- Parallelization and performance enhancements:
    - `num_workers=8`: Enables multi-process data loading.
    - `pin_memory=True`: Speeds up data transfer to GPU.
    - `prefetch_factor=4`: Loads data in advance to reduce I/O wait.
    - `non_blocking=True`: Ensures faster GPU data transfers.
    - `torch.set_num_threads(os.cpu_count())`: Fully utilizes available CPU cores.
    - `PYTORCH_CUDA_ALLOC_CONF` set for better memory handling on CUDA.

These changes should significantly improve data loading throughput and reduce GPU idle time.

#### Turn this into a Distributed Training Script 
The script has now been fully converted for Distributed Data Parallel (DDP) training using PyTorch's `torch.distributed` module.
- Key Features Added:
    - Uses `torch.multiprocessing.spawn` to launch multiple processes across available GPUs.
    - Leverages `DistributedSampler` to partition the dataset across processes.
    - Wraps the model with `DistributedDataParallel` (DDP) for synchronized training.
    - Logs metrics to TensorBoard only from rank 0 to avoid duplication.
    - Ensures each epoch sees the full dataset using `train_sampler.set_epoch(epoch)`.

#### Save Checkpoints for Resuming Later.
- Checkpointing is now built into the distributed training script.
- What was added:
    - `save_checkpoint()`: Saves model, optimizer, and epoch state to a .pth file.
    - `load_checkpoint()`: Loads from checkpoint if it exists, and resumes training.
    - Per-rank checkpointing: Each process saves its own checkpoint (e.g., `checkpoint_rank0.pth`).
- Usage:
    - If you stop training, just re-run the script—it will pick up from the last saved epoch.
    
**Note:** Make sure you save the code below as a `.py` file. Also note that we eliminate the `matplotlib` plotting functions from this final code. This is because `matplotlib` will not work in a `.py` file. However, we can save the output to file so that we can plot them separately later.

In [ ]:
# Load packages
import os
import time
import torch
import datetime
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.distributed as dist
import torch.multiprocessing as mp
from sklearn.datasets import make_classification
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, Dataset, DistributedSampler

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.set_num_threads(os.cpu_count())

def setup(rank, world_size):
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)

def cleanup():
    dist.destroy_process_group()

# Wrap the NumPy arrays in a PyTorch data set for use with DataLoader.
class LargeDataset(Dataset): 
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define a massive deep neural network.
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(200, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            *[layer for _ in range(40) for layer in [
                nn.Linear(2048, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
                nn.Dropout(0.2)
            ]],
            nn.Linear(2048, 2)
        )

    def forward(self, x):
        return self.net(x)

def save_checkpoint(state, filename="checkpoint.pth"):
    torch.save(state, filename)

def load_checkpoint(model, optimizer, filename="checkpoint.pth"):
    if os.path.isfile(filename):
        checkpoint = torch.load(filename, map_location="cpu")
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Loaded checkpoint from '{filename}' at epoch {start_epoch}")
        return start_epoch
    else:
        print(f"No checkpoint found at '{filename}', starting from scratch.")
        return 0

def train(rank, world_size):
    setup(rank, world_size)

    device = torch.device(f"cuda:{rank}")

    # Create 10 million synthetic dataset with 200 feature in total: 150 informative features and 30 redundant/correlated features. The remain. 
    X, y = make_classification(n_samples=10e6, n_features=200, n_informative=150,
                               n_redundant=30, n_classes=2, random_state=42)
    # Split the dataset into 90% training and 10% testing.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    train_dataset = LargeDataset(X_train, y_train)
    test_dataset = LargeDataset(X_test, y_test)

    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    test_sampler = DistributedSampler(test_dataset, num_replicas=world_size, rank=rank, shuffle=False)

    # Load data in batches of 2048. num_workers=4 means parallel data loading with 4 processes (if supported).
    train_loader = DataLoader(train_dataset, batch_size=2048, sampler=train_sampler,
                              num_workers=8, pin_memory=True, prefetch_factor=4)
    test_loader = DataLoader(test_dataset, batch_size=2048, sampler=test_sampler,
                             num_workers=8, pin_memory=True, prefetch_factor=4)

    model = DeepNN().to(device)
    model = DDP(model, device_ids=[rank])

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

    # Setup TensorBoard logging
    log_dir = f"runs/ddp_rank{rank}_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    writer = SummaryWriter(log_dir=log_dir) if rank == 0 else None

    num_epochs = 50
    checkpoint_file = f"checkpoint_rank{rank}.pth"
    start_epoch = load_checkpoint(model, optimizer, checkpoint_file)

    # Time measurement
    start_time = time.time()

    # Training loop
    for epoch in range(start_epoch, num_epochs):
        model.train()
        train_sampler.set_epoch(epoch)
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total

        # TensorBoard logging step
        if rank == 0:
            writer.add_scalar("Loss/train", avg_loss, epoch)
            writer.add_scalar("Accuracy/val", accuracy, epoch)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Val Accuracy: {accuracy:.4f}")

            # Save checkpoint
            save_checkpoint({
                'epoch': epoch,
                'model_state_dict': model.module.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, filename=checkpoint_file)

    if rank == 0:
        writer.close()
        end_time = time.time()
        print(f"Training completed in {(end_time - start_time) / 3600:.2f} hours")

    cleanup()

if __name__ == "__main__":
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)

### Submitting Jobs on Borah

Since the code cannot run on our local machines or OnDemand, we will have to run it by submitting a job on Borah. Below is a sample script for submitting jobs on Borah. The codes below must be saved into a `.sh` file. 

```bash
#!/bin/bash
#Account and Email Information
#SBATCH -A tnde  ## User ID
#SBATCH --mail-type=end
#SBATCH --mail-user=titusnyarkonde@u.boisestate.edu
# Specify parition (queue)
#SBATCH --partition=bsudfq
# Join output and errors into output.
#SBATCH -o test_borah.o%j
#SBATCH -e test_borah.e%j
# Specify job not to be rerunable.
#SBATCH --no-requeue
# Job Name.
#SBATCH --job-name="test_borah_login2"
# Specify walltime.
#SBATCH -t 06-23:59:59 
# ###SBATCH --time=48:00:00
# Specify number of requested nodes.
#SBATCH -N 1
# Specify the total number of requested procs:
#SBATCH -n 48
# number of cpus per task
#SBATCH --cpus-per-task=1 
# Number of GPUs per node.
# #SBATCH --gres=gpu:1
# load all necessary modules and ctivate the conda environment
module load slurm
module load gcc/7.5.0
module load gsl/gcc8/2.6
module load openmpi/gcc/64/1.10.7
module load cuda11.0/toolkit/11.0.3 
source /bsuhome/tnde/miniconda3/etc/profile.d/conda.sh
conda activate /bsuhome/tnde/miniconda3/envs/ml_tutorial
# Echo commands to stdout (standard output).
# set -x
# Copy your code & data to your R2 Home directory using
# the SFTP (secure file transfer protocol).
# Go to the directory where the actual BATCH file is present.
cd /bsuhome/tnde/ondemand/dev/ood_tutorials/template
 # The �python� command runs your python code.
# All output is dumped into test_borah.o%j with �%j� replaced by the Job ID.
## The file Multiprocessing.py must also 
## be in $/home/tnde/P1_Density_Calibration/Density3D
mpirun -np 8 python3 ml_benchmarking.py >>log.out
# python3 demo_analytic.py >>log.out
```

Now, we will run the code under different scenarios and report the time it took to finish running. Remember that this tutorial is about benchmarking the time the code takes to run. This means we are not interested in the model's performance and accuracy. We are only interested in the time it takes to finish running the code. Hence, we will not be showing training and validation accuracy/loss curves here.